In [1]:
import os
import re
from collections import defaultdict

import textract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from PyPDF2 import PdfReader, PdfWriter
from tqdm.autonotebook import tqdm
import pandas as pd

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_5542/945261966.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
pdfs = []
for root, subdirs, files in os.walk("eng"):
    if len(files) == 0:
        continue
    for file in files:
        if file.lower().endswith(".pdf") or file.lower().endswith("pdf_"):
            year_search = re.search(r"\d{4}", file)
            year = int(year_search.group(0)) if year_search else None
            pdfs.append((os.path.join(root, file), year))
        else:
            print(file)
len(pdfs)

paragrapghs_with_topisc_sample.xlsx
paragrapghs_with_topisc_sample.csv
Banco-BPM-HDT-Q3-2023.xlsx
Banco-BPM-HDT-Q3-2023.xlsx
EDPR_AnnualESGKeyData.xlsx
thewholedata.xlsx
EDPR_AnnualESGKeyData.xlsx
Capgemini_-_2023-07-05_-_Capgemini_ESG Reporting.xlsx
Softtek-2019.pdf.Esp


806

In [3]:
stopwords_eng = stopwords.words("english")
morph = MorphAnalyzer()
RE_ENG_TEXT = re.compile(r"[a-zA-Z0-9\-]+")
RE_DERIVED = re.compile(r"\w+( -|- |-|! - )\w+")

In [4]:
def parse_pages_to_texts(file_name: str, year: int | None = None) -> list[dict[str, str]] | None:
    try:
        report = textract.process(file_name)
    except Exception as e:
        print(file, e)
        return None
    all_paragraphs = []
    for i, paragraph in enumerate(report.decode().split("\n\n")):
        paragraph = paragraph.replace("\n", " ").lower()
        report_page = RE_DERIVED.sub("", paragraph)
        report_page_lst = word_tokenize(report_page)
        tokens = []
        for word_ in filter(RE_ENG_TEXT.match, report_page_lst):
            if word_ and word_ not in stopwords_eng:
                word_ = word_.strip()
                word_ = morph.parse(word_)[0]
                normal_form = word_.normal_form
                tokens.append(normal_form)
        all_paragraphs.append(
            {
                "file_name": file_name,
                "year": year,
                "paragraph": i,
                "original_text": paragraph,
                "cleaned_text": " ".join(tokens),
            }
        )
    return all_paragraphs

In [5]:
df = []
for file, year in tqdm(pdfs):
    res = parse_pages_to_texts(file, year)
    if res:
        df.extend(res)

  0%|          | 0/806 [00:00<?, ?it/s]

The command `pdftotext eng/ESG Reports/Italy, France and Spain/ESG data/Unconfirmed 762891.crdownload.pdf -` failed with exit code 1
------------- stdout -------------
b''------------- stderr -------------
b"Syntax Error: Couldn't find trailer dictionary\nSyntax Error: Couldn't find trailer dictionary\nSyntax Error: Couldn't read xref table\n"
The filename extension .pdf_ is not yet supported by
textract. Please suggest this filename extension here:

    https://github.com/deanmalmgren/textract/issues

Available extensions include: .csv, .doc, .docx, .eml, .epub, .gif, .htm, .html, .jpeg, .jpg, .json, .log, .mp3, .msg, .odt, .ogg, .pdf, .png, .pptx, .ps, .psv, .rtf, .tab, .tff, .tif, .tiff, .tsv, .txt, .wav, .xls, .xlsx

The filename extension . pdf_ is not yet supported by
textract. Please suggest this filename extension here:

    https://github.com/deanmalmgren/textract/issues

Available extensions include: .csv, .doc, .docx, .eml, .epub, .gif, .htm, .html, .jpeg, .jpg, .json, .log,

In [6]:
df = pd.DataFrame(df)
df.head()

,file_name,year,paragraph,original_text,cleaned_text
0,eng/ESG Reports/BP - 2022.pdf,2022.0,0,reimagining energy for people and our planet b...,reimagining energy people planet bp sustainabi...
1,eng/ESG Reports/BP - 2022.pdf,2022.0,1,sustainability at bp,sustainability bp
2,eng/ESG Reports/BP - 2022.pdf,2022.0,2,our approach,approach
3,eng/ESG Reports/BP - 2022.pdf,2022.0,3,getting to net zero,getting net zero
4,eng/ESG Reports/BP - 2022.pdf,2022.0,4,improving people’s lives,improving people lives


In [7]:
df.shape

(2473479, 5)

In [8]:
df.to_csv("paragraphs_eng.csv.zip")